In [ ]:
# Install prerequisites

# Install PyCocoTools needed for FasterRCNN
!pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

# Copy useful functions from pytorch vision tools
%cp ../input/pytorch-vision-tools/references/detection/*.* .

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import models
from os import path
import torch
import torchvision.utils
import utils
from engine import train_one_epoch, evaluate
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import Adam
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pickle
import time
import matplotlib.pyplot as plt
import tqdm
from sklearn.model_selection import train_test_split
import os
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import recall_score, precision_score
import copy

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Load the training data csv file
data_dir = "../input/vinbigdata1024mlp/"

df_test = pd.DataFrame(columns=['image_id', 'prediction_string'])

df_test['image_id'] = os.listdir('../input/vinbigdata1024mlp/test')
# Remove images that do not contain any anomolies
#df_train = df_train[df_train["class_id"]!=14]

# Print out the total number of images and the total number of annotations 
print(f"{df_test['image_id'].nunique()} images")
print(f"{len(df_test.index)} annotations")

In [ ]:
# Define a dataset class which defines how to load images,targets for training and validation
class VinBigDataset_test(Dataset):
    def __init__(self, img_dir, df, transform):
        self.img_dir = img_dir
        self.df = df
        self.imgs = df["image_id"]
        self.transform = transform
        
    def __len__(self):
        # Return the number of elements in the dataset
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # The dataset iterates over each image id
        # Return the requested image,target from the dataset
        
        img_file_name = self.imgs.iloc[idx]
        img = np.asarray(Image.open(os.path.join(self.img_dir, img_file_name)).convert("RGB"))
        

        if self.transform is not None:
            img = self.transform(image=img)['image']
            
        img = torch.Tensor(img)
        

        return img, img_file_name

In [ ]:
def create_augmentations(train):
    if train:
        return A.Compose([
                
                A.RandomCrop(height=800,width=800),
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.2, rotate_limit=15),
                A.RandomBrightnessContrast(p=0.4),
                A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
                ToTensorV2(p=1.0)
                    
        ])
    else:
        return A.Compose([
             A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)
            
        ])

In [ ]:
# Create an instance of the dataset and test dataloader

test_img_dir = path.join(data_dir, 'test')

test_dataset = VinBigDataset_test(test_img_dir, df_test, create_augmentations(train=False))


data_loader_test = DataLoader(\
    test_dataset, batch_size=1, shuffle=False, num_workers=4)


print(f"{len(test_dataset)} items in the test set")

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
            
def initialize_model_test(num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    model_name = "inception"
    """ Inception v3
    Be careful, expects (299,299) sized images and has auxiliary output
    """
    model_ft = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 299

    return model_ft, input_size

In [ ]:
model, _ = initialize_model_test(2, feature_extract=False)
model.load_state_dict(torch.load(WEIGHTS_FILE))
model.eval()

In [ ]:


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


WEIGHTS_FILE = "../input/classifier-weights/classification_model_weights.bin"



model = model.to(device)

In [ ]:
row_no = 0
for img, img_file_name in data_loader_test:
    img= img.to(device)
    
    outputs = model(img)
    _, pred = torch.max(outputs, 1)
    if pred == 0:
        df_test.iloc[row_no]['prediction_string'] = '14 1 0 0 1 1'

    row_no += 1

In [ ]:
df_test['prediction_string'].value_counts()

In [ ]:
remaining_imgs_df = df_test[pd.isnull(df_test).any(axis=1)]

In [ ]:
remaining_test_dataset = VinBigDataset_test(test_img_dir, remaining_imgs_df, create_augmentations(train=False))


remaining_data_loader_test = DataLoader(\
    remaining_test_dataset, batch_size=1, shuffle=False, num_workers=4)



In [ ]:
# Function to create an instance of the model
def create_model(weights_filepath):
    # The model has 14 classes
    num_classes = 14
    
    # Use resnet50 pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        
    # Fetch the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    model.load_state_dict(torch.load(weights_filepath, map_location=device))
    
   
    return model

In [ ]:
abnormality_model = create_model("../input/abnormality-detection-weights/model-weights.bin")
abnormality_model.to(device)
abnormality_model.eval()

In [ ]:
iteration = 0
for images, img_file_name in remaining_data_loader_test:

    images = images.to(device)

    output = abnormality_model(images)
    if iteration == 2:
        print(output)
        break
    iteration += 1

In [ ]:
torch.save(model_ft.state_dict(), f"classification_model_weights.bin")
pickle.dump(train_stats, open(f"train_stats.pkl", 'wb'))
pickle.dump(val_stats, open(f"val_stats.pkl", 'wb'))

In [ ]:
submission_df = df_test.copy()

In [ ]:
submission_df['prediction_string'] = "14 1 0 0 1 1"

In [ ]:
submission_df
